## Download the dataset - from Kaggle

Skip this step if you have the data in Drive already!

In [5]:
!pip install -q kaggle

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 441, in run
    conflicts = self._determine_conflicts(to_install)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 

In [6]:
from google.colab import files
files.upload()

KeyboardInterrupt: 

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets list

In [ ]:
! kaggle competitions download -c bgu-recommendation-systems-click-through-rate

In [ ]:
import zipfile
import os

zip_file_path = 'bgu-recommendation-systems-click-through-rate.zip'
extracted_folder_path = 'bgu-recommendation-systems-click-through-rate'

# Create the extraction directory if it doesn't exist
os.makedirs(extracted_folder_path, exist_ok=True)

# Open the zip file and extract its contents
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)

print(f"Successfully extracted contents to {extracted_folder_path}")

## Download the dataset - from Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
import os
import pandas as pd
from tqdm import tqdm

parent_dir = '/content/drive/MyDrive/cs247project/data/train_data'

# Initialize an empty DataFrame to store the combined data
df = pd.DataFrame()

# Iterate through each folder (train1 through train7)
br = 0
br2 = 0
for folder_name in os.listdir(parent_dir):
    if br == 1:
      break
    folder_path = os.path.join(parent_dir, folder_name)
    br = 1

    # Check if the path is a directory
    if os.path.isdir(folder_path):
        # Iterate through each CSV file in the folder
        for file_name in tqdm(os.listdir(folder_path), desc=f"Processing {folder_name}"):
            if file_name.endswith('.csv'):

                file_path = os.path.join(folder_path, file_name)

                # Read the CSV file into a DataFrame
                this_df = pd.read_csv(file_path)

                # Concatenate the current DataFrame with the combined_data DataFrame
                df = pd.concat([df, this_df], ignore_index=True)

# Print the combined DataFrame
print(df)


Processing train1: 100%|██████████| 11/11 [01:44<00:00,  9.52s/it]


         page_view_start_time  \
0               1586392478339   
1               1587264330576   
2               1586971453279   
3               1586446676383   
4               1587072791703   
...                       ...   
4867448         1586283443662   
4867449         1587236723732   
4867450         1586865909283   
4867451         1586273456619   
4867452         1587268134242   

                                              user_id_hash  \
0        939e2ccfb21e0ec9b0787c394dea1ae935369da0bcc597...   
1        decc9bcd3a7f2807721a35368b362782ec5bf4b4fb7ecf...   
2        30b6851a72026b837c7b33190ec7684763f36a8cd7722c...   
3        0b03f7d7bfc9d81e55ebeb994d572ca65cca353e7aaf85...   
4        b46cb1c9681e58c0a09b88b82c36913cbf3f710674eb39...   
...                                                    ...   
4867448  0d3c92f25f9375068a1b9215e86da67e0f09283e145af8...   
4867449  7ceacd0153187e3e4a0e398278c3b82e6c5d0eba4fdf1f...   
4867450  7a91b4737b70b51bc97da948df6c45db8f9f

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set(color_codes=True)

## Get basic info about features

In [4]:
df.head(5)

""


In [5]:
df.dtypes

Series([], dtype: object)

In [6]:
df.shape

(0, 0)

In [7]:
df.count()

Series([], dtype: int64)

In [8]:
# Visualize is_click, make sure the classes are balanced
plt.figure(figsize=(8, 5))
sns.countplot(x='is_click', data=df)
plt.title('Distribution of Clicks')
plt.xlabel('is_click')
plt.ylabel('Count')
plt.show()

ValueError: Could not interpret value `is_click` for `x`. An entry with this name does not appear in `data`.

<Figure size 800x500 with 0 Axes>

## Remove null values

In [ ]:
print(df.isnull().sum())

In [ ]:
df = df.dropna()    # Dropping the missing values.
df.count()

In [ ]:
print(df.isnull().sum())

## Check for outliers

In [ ]:
cols = ['page_view_start_time', 'empiric_clicks', 'user_recs', 'time_of_day'] # can add/remove from this as needed

for col in cols:
      # Create a boxplot for the current column
      sns.boxplot(x=df[col])

      # Set plot title with the column name
      plt.title(f'Boxplot for {col}')

      # Show the plot
      plt.show()

In [ ]:
unique_user_ids = df['user_id_hash'].nunique()
print(unique_user_ids)

In [ ]:
target_id_hash = df['target_id_hash'].nunique()
print(target_id_hash)

## Identify most important features:

In [12]:
import torch

from spotlight.factorization.explicit import ExplicitFactorizationModel
from spotlight.datasets.movielens import get_movielens_dataset


model = ExplicitFactorizationModel(loss='regression',
                                   embedding_dim=128,  # latent dimensionality
                                   n_iter=10,  # number of epochs of training
                                   batch_size=1024,  # minibatch size
                                   l2=1e-9,  # strength of L2 regularization
                                   learning_rate=1e-3,
                                   use_cuda=torch.cuda.is_available())


In [13]:
from spotlight.cross_validation import random_train_test_split
from spotlight.interactions import Interactions
import numpy as np

# Assuming 'user_id_hash' is your user ID column and 'target_id_hash' is your item ID column
# and 'is_click' is your interaction/rating column (you may need to convert these to integer indices)
user_ids = df['user_id_hash'].astype('category').cat.codes.values
item_ids = df['target_id_hash'].astype('category').cat.codes.values
ratings = df['is_click'].values

# Create an Interactions object
interactions = Interactions(user_ids, item_ids, ratings)

# Now you can split it using spotlight's function

train, test = random_train_test_split(interactions, random_state=np.random.RandomState(42))

print('Split into \n {} and \n {}.'.format(train, test))

Split into 
 <Interactions dataset (3950069 users x 61763 items x 3893962 interactions)> and 
 <Interactions dataset (3950069 users x 61763 items x 973491 interactions)>.


In [1]:
model.fit(train, verbose=True)

NameError: name 'model' is not defined

In [ ]:
from spotlight.evaluation import rmse_score

train_rmse = rmse_score(model, train)
test_rmse = rmse_score(model, test)

print('Train RMSE {:.3f}, test RMSE {:.3f}'.format(train_rmse, test_rmse))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

numeric_features = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
df_numeric_features_only = df.loc[:, numeric_features]

X = df_numeric_features_only.drop('is_click', axis=1)
y = df_numeric_features_only['is_click']

# Create and train the RandomForestClassifier
model = RandomForestClassifier(random_state=42)
model.fit(X, y)

# Get feature importances
feature_importances = model.feature_importances_

# Combine with feature names
feature_importance_dict = dict(zip(X.columns, feature_importances))

# Sort and display
sorted_features = sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)
print(sorted_features)



In [ ]:
# Plot the two most important features
plt.figure(figsize=(10, 6))

# Scatter plot with custom colors based on is_click
colors = df['is_click'].map({0: 'red', 1: 'green'})
plt.scatter(df['empiric_calibrated_recs'], df['empiric_clicks'], c=colors)

# Adding labels and title
plt.title('Scatter Plot: empiric_calibrated_recs vs empiric_clicks')
plt.xlabel('empiric_calibrated_recs')
plt.ylabel('empiric_clicks')

plt.show()

In [ ]:
# Plot the some of the less important features
plt.figure(figsize=(10, 6))

# Scatter plot with custom colors based on is_click
colors = df['is_click'].map({0: 'red', 1: 'green'})
plt.scatter(df['user_target_recs'], df['user_recs'], c=colors)

# Adding labels and title
plt.title('Scatter Plot: user_target_recs vs user_recs')
plt.xlabel('user_target_recs')
plt.ylabel('user_recs')

plt.show()


In [ ]:
# https://www.kaggle.com/code/imoore/intro-to-exploratory-data-analysis-eda-in-python

## Baseline Classifiers

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# All our baselines only use the numerical features in the dataset
X = df_numeric_features_only.drop('is_click', axis=1)  # Features
y = df_numeric_features_only['is_click']  # Target variable

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

# Create a Logistic Regression model
model = LogisticRegression(random_state=42)

# Train the model on the training set
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}")

### Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn import tree

# Create a decision tree classifier
clf = DecisionTreeClassifier(random_state=42)

# Train the classifier on the training set
clf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}")

### Random Forest

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

# Create a Random Forest classifier
clf = RandomForestClassifier(random_state=42)

# Train the classifier on the training set
clf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}")


### Perceptron

In [ ]:
import tensorflow as tf
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define one-layer neural network
model = tf.keras.Sequential([
    tf.keras.layers.Dense(1, activation='sigmoid', input_shape=(X_train_scaled.shape[1],))
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_scaled, y_train, epochs=10, batch_size=32, validation_data=(X_test_scaled, y_test))

# Evaluate the model
y_pred_probs = model.predict(X_test_scaled)
y_pred = (y_pred_probs > 0.5).astype(int)

accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}")
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", classification_rep)
